# SVM算法 

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import collections

In [2]:
filename = './NSL-KDD/KDDTrain+.txt'
# file = open(filename)
# lines = file.readlines()
df = pd.DataFrame(pd.read_table(filename,header=None,encoding='gb2312',sep=','))
# lines[0]
# dir(file)
df

E:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead.
  after removing the cwd from sys.path.


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
6,0,tcp,private,S0,0,0,0,0,0,0,...,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
7,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
9,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21


In [3]:
attack_dict = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}

In [4]:

arr = [1,2,3,41]
dics = {}
df[41]  = df[41].map(attack_dict)
num_dict = {}

In [5]:
df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,DoS,21
6,0,tcp,private,S0,0,0,0,0,0,0,...,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS,21
7,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,DoS,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,...,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,DoS,21
9,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,DoS,21


In [6]:
for i in arr:
    
    count_dic = dict(collections.Counter(df.iloc[:,i]))
    
#     print(count_dic)
    dics[i] = dict(zip(list(count_dic.keys()),range(len(list(count_dic.keys())))))
    num_dict[i] = len(list(count_dic))
#     print(count_dic)
    df[i] = df[i].map(dics[i])
print(num_dict)

{1: 3, 2: 70, 3: 11, 41: 5}


In [7]:
np_data = np.array(df)
ohe_arr = [1,2,3]
np_lab = np_data[:,-2:]
np_data = np_data[:,:-2]

for i in ohe_arr:
    ohe = OneHotEncoder()
    ohe.fit(list(map(lambda x:[x],range(num_dict[i]))))
    np_data = np.delete(np_data,i,axis=1)
    oh = ohe.transform(list(map(lambda x:[x],list(df[i])))).toarray()
    oh = np.array(oh)
    print(oh.shape)
    np_data = np.c_[np_data,oh]
np_data = np.c_[np_data,np_lab]

E:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
E:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

(125973, 3)
(125973, 70)


E:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(125973, 11)


In [8]:
np_data.shape

(125973, 124)

In [9]:
need_norm = [ 0,  1,  2,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 16,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37]

for i in need_norm:
    if np.std(np_data[:,i]) != 0:
        np_data[:,i] = (np_data[:,i] - np.mean(np_data[:,i]))/np.std(np_data[:,i])
    

In [10]:
# np_data.shape
# pd.DataFrame(np_data)

,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,-0.110249,-0.792078,-0.007679,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
1,-0.110249,-0.713710,-0.007737,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
2,-0.110249,-0.635342,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0
3,-0.110249,-0.556973,-0.007723,0.0,-0.089486,-0.007736,-0.095076,-0.027023,1.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
4,-0.110249,-0.556973,-0.007728,0.0,-0.089486,-0.007736,-0.095076,-0.027023,1.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
5,-0.110249,-0.635342,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0
6,-0.110249,-0.635342,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0
7,-0.110249,-0.635342,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0
8,-0.110249,-0.478605,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0
9,-0.110249,-0.635342,-0.007762,0.0,-0.089486,-0.007736,-0.095076,-0.027023,0.0,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0


In [11]:
from sklearn.svm import SVC
import datetime

In [12]:
print('START')
start = datetime.datetime.now()
svm = SVC(C=5000.0, kernel='rbf', gamma=0.001)
x = pd.DataFrame(np_data[:,:-2])
y = np_data[:,-2]


# uuu = np.isnan(x)
svm.fit(x,y[:])
end = datetime.datetime.now()
t = end-start
print('END')
print(t.seconds)
# x.dtype


START
END
71


In [15]:
y_hat = svm.predict(x)


In [17]:
np.sum(y == y_hat)/len(y)

0.9967453343176712